# Use ToolKits with OpenAI Functions

This notebook shows how to use the OpenAI functions agent with arbitrary toolkits.

In [14]:
from langchain import (
    LLMMathChain,
    OpenAI,
    SerpAPIWrapper,
    SQLDatabase,
    SQLDatabaseChain,
)
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.schema import SystemMessage

Load the toolkit

In [15]:
db = SQLDatabase.from_uri("sqlite:///../../../../../notebooks/Chinook.db")
toolkit = SQLDatabaseToolkit(llm=ChatOpenAI(), db=db)

Set a system message specific to that toolkit

In [16]:
agent_kwargs = {
    "system_message": SystemMessage(content="You are an expert SQL data analyst.")
}

In [17]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
agent = initialize_agent(
    toolkit.get_tools(), 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    agent_kwargs=agent_kwargs,
)

In [219]:
from pydantic import BaseModel, Field
from typing import List, Union, Tuple, Literal

In [267]:
class Search(BaseModel):
    query: str
        
class PythonREPL(BaseModel):
    query: str
    
class ToolCall(BaseModel):
    tool_name: Literal['PythonREPL', 'Search']
    actions: Union[PythonREPL, Search]
        
class ToolSelection(BaseModel):
    """A list of actions to take."""
    
    actions: List[ToolCall]

In [268]:
from typing import Any, Dict


def _resolve_schema_references(schema: Any, definitions: Dict[str, Any]) -> Any:
    """
    Resolves the $ref keys in a JSON schema object using the provided definitions.
    """
    if isinstance(schema, list):
        for i, item in enumerate(schema):
            schema[i] = _resolve_schema_references(item, definitions)
    elif isinstance(schema, dict):
        if "$ref" in schema:
            ref_key = schema.pop("$ref").split("/")[-1]
            ref = definitions.get(ref_key, {})
            schema.update(ref)
        else:
            for key, value in schema.items():
                schema[key] = _resolve_schema_references(value, definitions)
    return schema

In [269]:
openai_schema = ToolSelection.schema()

In [270]:
openai_schema = ToolSelection.schema()
openai_schema = _resolve_schema_references(
    openai_schema, openai_schema["definitions"]
)

In [271]:
def _convert_schema(schema: dict) -> dict:
    props = {k: {"title": k, **v} for k, v in schema["properties"].items()}
    return {
        "type": "object",
        "properties": props,
        "required": schema.get("required", []),
    }

In [272]:
tool_selection = {
    "name": openai_schema["title"],
    "description": openai_schema["description"],
    "parameters": openai_schema,
}

In [273]:
del tool_selection['parameters']['definitions']

In [274]:
tool_selection

{'name': 'ToolSelection',
 'description': 'A list of actions to take.',
 'parameters': {'title': 'ToolSelection',
  'description': 'A list of actions to take.',
  'type': 'object',
  'properties': {'actions': {'title': 'Actions',
    'type': 'array',
    'items': {'title': 'ToolCall',
     'type': 'object',
     'properties': {'tool_name': {'title': 'Tool Name',
       'enum': ['PythonREPL', 'Search'],
       'type': 'string'},
      'actions': {'title': 'Actions',
       'anyOf': [{'title': 'PythonREPL',
         'type': 'object',
         'properties': {'query': {'title': 'Query', 'type': 'string'}},
         'required': ['query']},
        {'title': 'Search',
         'type': 'object',
         'properties': {'query': {'title': 'Query', 'type': 'string'}},
         'required': ['query']}]}},
     'required': ['tool_name', 'actions']}}},
  'required': ['actions']}}

In [275]:
def _convert(f):
    return {
        **{'title': f['name']},
        **f['parameters']
    }

In [276]:
tool_selection1 = {'name': 'ToolSelection',
 'description': 'A list of actions to take.',
 'parameters': {'title': 'ToolSelection',
  'description': 'A list of actions to take.',
  'type': 'object',
  'properties': {'actions': {'title': 'Actions',
    'type': 'array',
    'items': {'anyOf': [_convert(f) for f in agent.agent.functions]}}},
  'required': ['actions']}}

In [277]:
from langchain.schema import HumanMessage

In [278]:
result = agent.agent.llm.predict_messages(
   [HumanMessage(content="Use the following tools as neccesary to answer the question"), 
    HumanMessage(content="who is leo dicaprio dating? how old is leo?")] ,
    functions=[tool_selection],
)

In [279]:
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ToolSelection', 'arguments': '{\n  "actions": [\n    {\n      "tool_name": "Search",\n      "actions": {\n        "query": "Leo DiCaprio current girlfriend"\n      }\n    },\n    {\n      "tool_name": "Search",\n      "actions": {\n        "query": "Leo DiCaprio age"\n      }\n    }\n  ]\n}'}}, example=False)

In [ ]:
result = agent.agent.llm.predict_messages(
   [HumanMessage(content="Use the following tools as neccesary to answer the question"), 
    HumanMessage(content="what many record in the song db?")] ,
    functions=[tool_selection1],
)

In [202]:
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ToolSelection', 'arguments': '{\n  "actions": [\n    {\n      "name": "Google Search",\n      "query": "Leo DiCaprio current girlfriend"\n    },\n    {\n      "name": "Google Search",\n      "query": "Leo DiCaprio age"\n    }\n  ]\n}'}}, example=False)

In [280]:
import json

In [281]:
ToolSelection(**json.loads(result.additional_kwargs['function_call']['arguments']))

ToolSelection(actions=[ToolCall(tool_name='Search', actions=PythonREPL(query='Leo DiCaprio current girlfriend')), ToolCall(tool_name='Search', actions=PythonREPL(query='Leo DiCaprio age'))])

In [70]:
from langchain.llms import OpenAI, Cohere, Anthropic, HuggingFaceHub, GooglePalm
model = OpenAI()
model.predict("say hi")

'\n\nHi there!'

In [13]:
agent.run("how many different artists are there?")



> Entering new  chain...

Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(DISTINCT artist_name) AS num_artists FROM artists'}`


Error: (sqlite3.OperationalError) no such table: artists
[SQL: SELECT COUNT(DISTINCT artist_name) AS num_artists FROM artists]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Invoking: `sql_db_list_tables` with `{}`


MediaType, Track, Playlist, sales_table, Customer, Genre, PlaylistTrack, Artist, Invoice, Album, InvoiceLine, Employee
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(DISTINCT artist_id) AS num_artists FROM Artist'}`


Error: (sqlite3.OperationalError) no such column: artist_id
[SQL: SELECT COUNT(DISTINCT artist_id) AS num_artists FROM Artist]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(DISTINCT Name) AS num_artists FROM Artist'}`


[(275,)]There are 275 different artists in the database.

> Finished chain.


'There are 275 different artists in the database.'